# Scraping Data Harga Pangan dari BI (Bank Indonesia)

Notebook ini digunakan untuk melakukan scraping data harga pangan dari website Bank Indonesia.

**Target URL:**
- Base URL: `https://www.bi.go.id/hargapangan/TabelHarga/PasarTradisionalKomoditas`

**Target Data:**
- Komoditas: **Beras** (semua kualitas)
- Periode: Per bulan
- Cakupan: Seluruh provinsi Indonesia

**Catatan:** Scraping ini menggunakan BeautifulSoup dan Selenium untuk parsing HTML table.

## 1. Import Library yang Diperlukan

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from datetime import datetime
import time
import re

# Install jika belum ada:
# pip install requests beautifulsoup4 pandas lxml selenium

print("✓ Library berhasil di-import")

✓ Library berhasil di-import


## 2. Konfigurasi Scraping

In [2]:
# Konfigurasi URL dan target
BASE_URL = 'https://www.bi.go.id/hargapangan/TabelHarga/PasarTradisionalKomoditas'

# Target komoditas
TARGET_KOMODITAS = 'Beras'  # Semua jenis beras

# Headers untuk request (agar tidak diblokir sebagai bot)
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'id-ID,id;q=0.9,en-US;q=0.8,en;q=0.7',
    'Referer': 'https://www.bi.go.id/'
}

print(f"Target URL: {BASE_URL}")
print(f"Target Komoditas: {TARGET_KOMODITAS}")

Target URL: https://www.bi.go.id/hargapangan/TabelHarga/PasarTradisionalKomoditas
Target Komoditas: Beras


## 3. Fungsi untuk Scraping HTML Table

In [3]:
def scrape_bi_harga_pangan():
    """
    Fungsi untuk melakukan scraping data harga pangan dari BI

    Returns:
    - Dictionary berisi data hasil scraping
    """
    try:
        print(f"\n{'='*70}")
        print(f"Memulai scraping dari: {BASE_URL}")
        print(f"{'='*70}")

        # Kirim request ke website
        print("\n[1] Mengirim HTTP request...")
        response = requests.get(BASE_URL, headers=HEADERS, timeout=30)

        print(f"    Status Code: {response.status_code}")

        if response.status_code != 200:
            return {
                'success': False,
                'error': f'HTTP {response.status_code}',
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

        # Parse HTML dengan BeautifulSoup
        print("\n[2] Parsing HTML...")
        soup = BeautifulSoup(response.content, 'html.parser')

        # Cari semua table di halaman
        tables = soup.find_all('table')
        print(f"    Ditemukan {len(tables)} table di halaman")

        if not tables:
            return {
                'success': False,
                'error': 'Tidak ada table ditemukan',
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

        # Ekstrak data dari table
        print("\n[3] Mengekstrak data dari table...")
        all_data = []

        for idx, table in enumerate(tables):
            print(f"\n    Memproses table #{idx + 1}...")

            # Coba ekstrak dengan pandas (lebih mudah)
            try:
                df_list = pd.read_html(str(table))
                if df_list:
                    df = df_list[0]
                    print(f"      Shape: {df.shape}")
                    print(f"      Columns: {list(df.columns)[:5]}...")  # Tampilkan 5 kolom pertama

                    # Filter untuk Beras
                    if 'Komoditas' in df.columns or 'komoditas' in df.columns or any('Komoditas' in str(col) for col in df.columns):
                        # Cari kolom yang berisi nama komoditas
                        komoditas_col = None
                        for col in df.columns:
                            if 'komoditas' in str(col).lower():
                                komoditas_col = col
                                break

                        if komoditas_col:
                            # Filter baris yang mengandung "Beras"
                            df_beras = df[df[komoditas_col].str.contains('Beras', case=False, na=False)]
                            print(f"      Ditemukan {len(df_beras)} baris data Beras")

                            if not df_beras.empty:
                                all_data.append({
                                    'table_index': idx,
                                    'data': df_beras.to_dict('records')
                                })
                    else:
                        # Jika tidak ada kolom Komoditas, simpan semua data
                        print(f"      Tidak ada kolom Komoditas, menyimpan semua data")
                        all_data.append({
                            'table_index': idx,
                            'data': df.to_dict('records')
                        })

            except Exception as e:
                print(f"      Error parsing table #{idx + 1}: {str(e)}")

        print(f"\n[4] Total data yang berhasil diekstrak: {len(all_data)} table")

        return {
            'success': True,
            'total_tables': len(all_data),
            'data': all_data,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'source_url': BASE_URL
        }

    except requests.exceptions.Timeout:
        print(f"✗ Error: Request timeout")
        return {
            'success': False,
            'error': 'Timeout',
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

    except Exception as e:
        print(f"✗ Error: {str(e)}")
        return {
            'success': False,
            'error': str(e),
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

## 4. Jalankan Scraping

In [4]:
# Jalankan scraping
hasil_scraping = scrape_bi_harga_pangan()

print("\n" + "="*70)
print("SCRAPING SELESAI")
print("="*70)

if hasil_scraping['success']:
    print(f"\n✓ Berhasil scraping {hasil_scraping['total_tables']} table")
    print(f"  Timestamp: {hasil_scraping['timestamp']}")
else:
    print(f"\n✗ Scraping gagal")
    print(f"  Error: {hasil_scraping.get('error', 'Unknown error')}")


Memulai scraping dari: https://www.bi.go.id/hargapangan/TabelHarga/PasarTradisionalKomoditas

[1] Mengirim HTTP request...
    Status Code: 200

[2] Parsing HTML...
    Ditemukan 0 table di halaman

SCRAPING SELESAI

✗ Scraping gagal
  Error: Tidak ada table ditemukan


## 4a. Inspeksi HTML (Debug)

In [5]:
# Inspect HTML content untuk debug
response = requests.get(BASE_URL, headers=HEADERS, timeout=30)
soup = BeautifulSoup(response.content, 'html.parser')

# Cek apakah ada form, div, atau element lain yang menandakan data dinamis
print("=== INSPEKSI HTML ===\n")
print(f"Total <div>: {len(soup.find_all('div'))}")
print(f"Total <table>: {len(soup.find_all('table'))}")
print(f"Total <script>: {len(soup.find_all('script'))}")

# Cari element yang mengandung kata "beras" atau "komoditas"
print("\n=== MENCARI KATA KUNCI ===")
text_content = soup.get_text().lower()
if 'beras' in text_content:
    print("✓ Kata 'beras' ditemukan di halaman")
else:
    print("✗ Kata 'beras' TIDAK ditemukan")

if 'komoditas' in text_content:
    print("✓ Kata 'komoditas' ditemukan di halaman")
else:
    print("✗ Kata 'komoditas' TIDAK ditemukan")

# Tampilkan sebagian HTML (1000 karakter pertama)
print("\n=== SAMPLE HTML (1000 char) ===")
print(str(soup)[:1000])

=== INSPEKSI HTML ===

Total <div>: 90
Total <table>: 0
Total <script>: 49

=== MENCARI KATA KUNCI ===
✗ Kata 'beras' TIDAK ditemukan
✓ Kata 'komoditas' ditemukan di halaman

=== SAMPLE HTML (1000 char) ===

<!DOCTYPE html>

<html dir="ltr" lang="en-US">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="SemiColonWeb" name="author"/>
<!-- Stylesheets
    ============================================= -->
<link href="/hargapangan/assets/fonts/fonts.css" rel="stylesheet"/>
<link href="/hargapangan/css/bootstrap.css" rel="stylesheet" type="text/css"/>
<link href="/hargapangan/style.css" rel="stylesheet" type="text/css"/>
<link href="/hargapangan/css/swiper.css" rel="stylesheet" type="text/css"/>
<link href="/hargapangan/css/dark.css" rel="stylesheet" type="text/css"/>
<link href="/hargapangan/css/font-icons.css" rel="stylesheet" type="text/css"/>
<link href="/hargapangan/css/animate.css" rel="stylesheet" type="text/css"/>
<link href="/hargapangan/css

## 4b. Kesimpulan & Solusi

**Hasil Inspeksi:**
- ❌ Website BI.go.id menggunakan JavaScript untuk render data
- ❌ HTML statis tidak mengandung table (0 table ditemukan)
- ❌ Data "beras" tidak ada di HTML statis
- ✅ Halaman memuat 49 <script> tags (aplikasi web dinamis)

**Solusi:** 
Gunakan **Selenium WebDriver** untuk render JavaScript dan ekstrak data dari table yang di-generate secara dinamis.

---

## 5. Setup Selenium WebDriver

In [3]:
# Install selenium jika belum ada
# Uncomment line berikut jika perlu install:
# %pip install selenium

try:
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.chrome.options import Options
    print("✓ Selenium library berhasil di-import")
except ImportError as e:
    print(f"✗ Error importing Selenium: {e}")
    print("  Jalankan: %pip install selenium")

✓ Selenium library berhasil di-import


## 6. Fungsi Scraping dengan Selenium

In [4]:
def scrape_bi_dengan_selenium():
    """
    Scraping data harga pangan BI menggunakan Selenium
    untuk render JavaScript dan ekstrak table
    """
    driver = None

    try:
        print(f"\n{'='*70}")
        print(f"Memulai scraping dengan Selenium")
        print(f"{'='*70}")

        # Setup Chrome options
        print("\n[1] Setup Chrome WebDriver...")
        chrome_options = Options()
        chrome_options.add_argument('--headless')  # Run tanpa GUI
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument(f'user-agent={HEADERS["User-Agent"]}')

        # Initialize WebDriver
        print("    Membuka Chrome driver...")
        driver = webdriver.Chrome(options=chrome_options)
        driver.set_page_load_timeout(30)

        # Buka halaman
        print(f"\n[2] Membuka URL: {BASE_URL}")
        driver.get(BASE_URL)

        # Tunggu halaman load (tunggu sampe table muncul)
        print("\n[3] Menunggu table dimuat...")
        try:
            # Tunggu max 20 detik untuk table muncul
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.TAG_NAME, "table"))
            )
            print("    ✓ Table ditemukan!")
        except:
            print("    ⚠ Timeout menunggu table (mungkin tidak ada table)")

        # Tunggu tambahan untuk memastikan data termuat
        time.sleep(3)

        # Ambil HTML setelah JavaScript dirender
        print("\n[4] Mengekstrak HTML yang sudah dirender...")
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Cari semua table
        tables = soup.find_all('table')
        print(f"    Ditemukan {len(tables)} table")

        if not tables:
            # Coba cek apakah ada error atau redirect
            print("\n    DEBUG: Mencari form atau element alternatif...")
            forms = soup.find_all('form')
            divs_with_class = soup.find_all('div', class_=True)
            print(f"    - Forms: {len(forms)}")
            print(f"    - Divs dengan class: {len(divs_with_class)}")

            # Screenshot untuk debug (simpan ke file)
            screenshot_path = "../data/bi-screenshot-debug.png"
            driver.save_screenshot(screenshot_path)
            print(f"    - Screenshot disimpan: {screenshot_path}")

            return {
                'success': False,
                'error': 'Tidak ada table ditemukan setelah render JavaScript',
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'tables_found': 0
            }

        # Ekstrak data dari semua table
        print("\n[5] Mengekstrak data dari table...")
        all_data = []

        for idx, table in enumerate(tables):
            print(f"\n    Table #{idx + 1}:")
            try:
                # Gunakan pandas untuk parse table
                df_list = pd.read_html(str(table))
                if df_list:
                    df = df_list[0]
                    print(f"      Shape: {df.shape}")
                    print(f"      Columns: {list(df.columns)[:5]}...")

                    # Filter untuk Beras jika ada kolom komoditas
                    komoditas_col = None
                    for col in df.columns:
                        if 'komoditas' in str(col).lower() or 'commodity' in str(col).lower():
                            komoditas_col = col
                            break

                    if komoditas_col:
                        df_beras = df[df[komoditas_col].str.contains('Beras', case=False, na=False)]
                        print(f"      Ditemukan {len(df_beras)} baris Beras")

                        if not df_beras.empty:
                            all_data.append({
                                'table_index': idx,
                                'data': df_beras.to_dict('records')
                            })
                    else:
                        # Simpan semua jika tidak ada kolom komoditas
                        print(f"      Tidak ada kolom komoditas, menyimpan semua")
                        all_data.append({
                            'table_index': idx,
                            'data': df.to_dict('records')
                        })

            except Exception as e:
                print(f"      Error: {str(e)}")

        print(f"\n[6] Total data diekstrak: {len(all_data)} table")

        return {
            'success': True,
            'total_tables': len(all_data),
            'data': all_data,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'source_url': BASE_URL,
            'method': 'selenium'
        }

    except Exception as e:
        print(f"\n✗ Error: {str(e)}")
        import traceback
        traceback.print_exc()

        return {
            'success': False,
            'error': str(e),
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

    finally:
        # Tutup browser
        if driver:
            print("\n[7] Menutup browser...")
            driver.quit()
            print("    ✓ Browser ditutup")

## 7. Jalankan Scraping dengan Selenium

In [5]:
# Jalankan scraping dengan Selenium
hasil_selenium = scrape_bi_dengan_selenium()

print("\n" + "="*70)
print("SCRAPING SELENIUM SELESAI")
print("="*70)

if hasil_selenium['success']:
    print(f"\n✓ Berhasil scraping dengan Selenium")
    print(f"  Total tables: {hasil_selenium['total_tables']}")
    print(f"  Timestamp: {hasil_selenium['timestamp']}")
else:
    print(f"\n✗ Scraping gagal")
    print(f"  Error: {hasil_selenium.get('error', 'Unknown error')}")


Memulai scraping dengan Selenium

[1] Setup Chrome WebDriver...
    Membuka Chrome driver...

[2] Membuka URL: https://www.bi.go.id/hargapangan/TabelHarga/PasarTradisionalKomoditas

[3] Menunggu table dimuat...
    ✓ Table ditemukan!

[4] Mengekstrak HTML yang sudah dirender...
    Ditemukan 8 table

[5] Mengekstrak data dari table...

    Table #1:
      Shape: (20, 1)
      Columns: [0]...
      Tidak ada kolom komoditas, menyimpan semua

    Table #2:
      Shape: (20, 1)
      Columns: [0]...
      Tidak ada kolom komoditas, menyimpan semua

    Table #3:
      Error: Missing optional dependency 'html5lib'.  Use pip or conda to install html5lib.

    Table #4:
      Error: Missing optional dependency 'html5lib'.  Use pip or conda to install html5lib.

    Table #5:
      Shape: (2, 9)
      Columns: [0, 1, 2, 3, 4]...
      Tidak ada kolom komoditas, menyimpan semua

    Table #6:
      Shape: (2, 9)
      Columns: [0, 1, 2, 3, 4]...
      Tidak ada kolom komoditas, menyimpan semu

C:\Users\user\AppData\Local\Temp\ipykernel_25896\1364328218.py:82: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(str(table))
C:\Users\user\AppData\Local\Temp\ipykernel_25896\1364328218.py:82: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(str(table))
C:\Users\user\AppData\Local\Temp\ipykernel_25896\1364328218.py:82: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(str(table))
C:\Users\user\AppData\Local\Temp\ipykernel_25896\1364328218.py:82: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read 

    ✓ Browser ditutup

SCRAPING SELENIUM SELESAI

✓ Berhasil scraping dengan Selenium
  Total tables: 6
  Timestamp: 2026-02-19 21:33:08


## 8. Eksplorasi Data Hasil Selenium

In [6]:
# Tampilkan struktur data dari hasil Selenium
if hasil_selenium['success'] and hasil_selenium['data']:
    print("STRUKTUR DATA HASIL SCRAPING SELENIUM:")
    print("="*70)

    for item in hasil_selenium['data']:
        table_idx = item['table_index']
        data_records = item['data']

        print(f"\nTable #{table_idx + 1}:")
        print(f"  Jumlah record: {len(data_records)}")

        if data_records:
            print(f"  Keys: {list(data_records[0].keys())}")
            print(f"\n  Sample data (3 baris pertama):")

            df_sample = pd.DataFrame(data_records[:3])
            print(df_sample.to_string())
            print()
else:
    print("Tidak ada data untuk ditampilkan")

STRUKTUR DATA HASIL SCRAPING SELENIUM:

Table #1:
  Jumlah record: 20
  Keys: [0]

  Sample data (3 baris pertama):
                         0
0                    Beras
1   Beras Kualitas Bawah I
2  Beras Kualitas Bawah II


Table #2:
  Jumlah record: 20
  Keys: [0]

  Sample data (3 baris pertama):
        0
0    Aceh
1    Bali
2  Banten


Table #5:
  Jumlah record: 2
  Keys: [0, 1, 2, 3, 4, 5, 6, 7, 8]

  Sample data (3 baris pertama):
     0               1   2   3   4   5   6   7   8
0   No  Komoditas (Rp) NaN NaN NaN NaN NaN NaN NaN
1  NaN             NaN NaN NaN NaN NaN NaN NaN NaN


Table #6:
  Jumlah record: 2
  Keys: [0, 1, 2, 3, 4, 5, 6, 7, 8]

  Sample data (3 baris pertama):
     0               1             2             3             4             5             6             7             8
0   No  Komoditas (Rp)  11/ 02/ 2026  12/ 02/ 2026  13/ 02/ 2026  16/ 02/ 2026  17/ 02/ 2026  18/ 02/ 2026  19/ 02/ 2026
1  NaN             NaN           NaN           NaN           

In [7]:
# Ringkasan table yang ditemukan (tanpa detail data)
if hasil_selenium['success'] and hasil_selenium['data']:
    print("RINGKASAN TABLE YANG BERHASIL DIEXTRAK:")
    print("="*70)

    for item in hasil_selenium['data']:
        table_idx = item['table_index']
        data_records = item['data']

        if data_records:
            df = pd.DataFrame(data_records)
            print(f"\nTable #{table_idx + 1}:")
            print(f"  Shape: {df.shape}")
            print(f"  Columns: {list(df.columns)}")

    print("\n" + "="*70)
    print(f"Total: {len(hasil_selenium['data'])} table berhasil diekstrak")
else:
    print("Tidak ada data")

RINGKASAN TABLE YANG BERHASIL DIEXTRAK:

Table #1:
  Shape: (20, 1)
  Columns: [0]

Table #2:
  Shape: (20, 1)
  Columns: [0]

Table #5:
  Shape: (2, 9)
  Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8]

Table #6:
  Shape: (2, 9)
  Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8]

Table #7:
  Shape: (35, 9)
  Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8]

Table #8:
  Shape: (35, 9)
  Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8]

Total: 6 table berhasil diekstrak


In [8]:
# Fokus pada table terbesar (paling likely adalah data harga)
# Cari table dengan jumlah baris terbanyak
if hasil_selenium['success'] and hasil_selenium['data']:
    max_rows = 0
    largest_table_idx = None

    for item in hasil_selenium['data']:
        if len(item['data']) > max_rows:
            max_rows = len(item['data'])
            largest_table_idx = item['table_index']

    print(f"Table terbesar: Table #{largest_table_idx + 1} dengan {max_rows} baris\n")

    # Tampilkan table terbesar (kemungkinan data harga)
    for item in hasil_selenium['data']:
        if item['table_index'] == largest_table_idx:
            df_main = pd.DataFrame(item['data'])
            print(f"Shape: {df_main.shape}")
            print(f"Columns: {list(df_main.columns)}\n")
            print("10 baris pertama:")
            print(df_main.head(10))
            break
else:
    print("Tidak ada data")

Table terbesar: Table #7 dengan 35 baris

Shape: (35, 9)
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8]

10 baris pertama:
      0                          1      2      3      4      5      6      7  \
0     I             SEMUA PROVINSI  15800  15800  15800  15550  15350  15800   
1    II                       ACEH  15300  15300  15300      -  14900  15250   
2   III             SUMATERA UTARA  14900  14900  14900      -  14700  14850   
3    IV             SUMATERA BARAT  18300  18300  18300      -      -  18300   
4     V                       RIAU  16350  16350  16350      -      -  16350   
5    VI             KEPULAUAN RIAU  15450  15450  15550      -      -  15450   
6   VII                      JAMBI  15400  15400  15400      -      -  15400   
7  VIII                   BENGKULU  15450  15450  15450  15450  15450  15450   
8    IX           SUMATERA SELATAN  15550  15650  15650      -      -  15650   
9     X  KEPULAUAN BANGKA BELITUNG  15150  15150      -      -      -  15150   

      

In [9]:
# Periksa apakah ada table dengan header yang menjelaskan kolom-kolom
# Table lain mungkin berisi metadata (header)
print("MENCARI TABLE HEADER/METADATA:")
print("="*70)

for item in hasil_selenium['data']:
    table_idx = item['table_index']
    data_records = item['data']

    # Cari table kecil yang mungkin header (< 5 baris)
    if len(data_records) < 5 and len(data_records) > 0:
        df_temp = pd.DataFrame(data_records)
        print(f"\nTable #{table_idx + 1} (kemungkinan header/metadata):")
        print(f"Shape: {df_temp.shape}")
        print(df_temp)
        print()

MENCARI TABLE HEADER/METADATA:

Table #5 (kemungkinan header/metadata):
Shape: (2, 9)
     0               1   2   3   4   5   6   7   8
0   No  Komoditas (Rp) NaN NaN NaN NaN NaN NaN NaN
1  NaN             NaN NaN NaN NaN NaN NaN NaN NaN


Table #6 (kemungkinan header/metadata):
Shape: (2, 9)
     0               1             2             3             4  \
0   No  Komoditas (Rp)  11/ 02/ 2026  12/ 02/ 2026  13/ 02/ 2026   
1  NaN             NaN           NaN           NaN           NaN   

              5             6             7             8  
0  16/ 02/ 2026  17/ 02/ 2026  18/ 02/ 2026  19/ 02/ 2026  
1           NaN           NaN           NaN           NaN  



In [10]:
# Bandingkan table #7 dan #8 (kemungkinan komoditas berbeda)
print("PERBANDINGAN TABLE DATA:")
print("="*70)

for item in hasil_selenium['data']:
    table_idx = item['table_index']

    # Hanya tampilkan table besar (>= 30 baris)
    if len(item['data']) >= 30:
        df_temp = pd.DataFrame(item['data'])
        print(f"\nTable #{table_idx + 1}:")
        print(f"  Shape: {df_temp.shape}")
        print(f"  Baris pertama: {df_temp.iloc[0].tolist()}")
        print(f"  Baris kedua: {df_temp.iloc[1].tolist()}")

PERBANDINGAN TABLE DATA:

Table #7:
  Shape: (35, 9)
  Baris pertama: ['I', 'SEMUA PROVINSI', np.int64(15800), np.int64(15800), '15800', '15550', '15350', np.int64(15800), '15750']
  Baris kedua: ['II', 'ACEH', np.int64(15300), np.int64(15300), '15300', '-', '14900', np.int64(15250), '15250']

Table #8:
  Shape: (35, 9)
  Baris pertama: ['I', 'SEMUA PROVINSI', np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan)]
  Baris kedua: ['II', 'ACEH', np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan)]


## 9. Transform Data ke Format Frontend

In [11]:
def transform_bi_to_frontend(hasil_selenium):
    """
    Transform data BI harga beras ke format frontend

    Input: hasil_selenium dengan table data harian
    Output: Format kompatibel dengan frontend (monthly averages)
    """

    # Mapping nama provinsi ke kode
    PROVINCE_CODE_MAPPING = {
        'ACEH': '11', 'SUMATERA UTARA': '12', 'SUMATRA UTARA': '12',
        'SUMATERA BARAT': '13', 'SUMATRA BARAT': '13', 'RIAU': '14',
        'JAMBI': '15', 'SUMATERA SELATAN': '16', 'SUMATRA SELATAN': '16',
        'BENGKULU': '17', 'LAMPUNG': '18', 'KEPULAUAN BANGKA BELITUNG': '19',
        'BANGKA BELITUNG': '19', 'KEP. BANGKA BELITUNG': '19',
        'KEPULAUAN RIAU': '21', 'KEP. RIAU': '21',
        'DKI JAKARTA': '31', 'JAKARTA': '31',
        'JAWA BARAT': '32', 'JAWA TENGAH': '33',
        'DI YOGYAKARTA': '34', 'YOGYAKARTA': '34', 'D.I. YOGYAKARTA': '34',
        'JAWA TIMUR': '35', 'BANTEN': '36', 'BALI': '51',
        'NUSA TENGGARA BARAT': '52', 'NTB': '52',
        'NUSA TENGGARA TIMUR': '53', 'NTT': '53',
        'KALIMANTAN BARAT': '61', 'KALIMANTAN TENGAH': '62',
        'KALIMANTAN SELATAN': '63', 'KALIMANTAN TIMUR': '64',
        'KALIMANTAN UTARA': '65', 'SULAWESI UTARA': '71',
        'SULAWESI TENGAH': '72', 'SULAWESI SELATAN': '73',
        'SULAWESI TENGGARA': '74', 'GORONTALO': '75', 'SULAWESI BARAT': '76',
        'MALUKU': '81', 'MALUKU UTARA': '82', 'PAPUA': '94',
        'PAPUA BARAT': '91', 'PAPUA TENGAH': '95', 'PAPUA PEGUNUNGAN': '96',
        'PAPUA SELATAN': '97', 'PAPUA BARAT DAYA': '92'
    }

    # Cari table data utama (table terbesar)
    max_rows = 0
    main_table = None

    for item in hasil_selenium['data']:
        if len(item['data']) > max_rows:
            max_rows = len(item['data'])
            main_table = item['data']

    if not main_table:
        return None

    # Convert ke DataFrame
    df = pd.DataFrame(main_table)

    print(f"Processing {len(df)} rows from BI data...")
    print(f"Columns: {list(df.columns)}")

    # Asumsi struktur: kolom 0 = No, kolom 1 = Provinsi, kolom 2-8 = harga harian
    # Hitung rata-rata harga dari kolom 2-8

    transformed_data = []
    current_month = datetime.now().strftime('%b').lower()  # 'feb'
    current_year = str(datetime.now().year)  # '2026'

    for idx, row in df.iterrows():
        province_name = str(row[1]).strip().upper()

        # Skip "SEMUA PROVINSI"
        if 'SEMUA' in province_name:
            continue

        # Get province code
        province_code = PROVINCE_CODE_MAPPING.get(province_name)

        if not province_code:
            print(f"  ⚠ Province not mapped: {province_name}")
            continue

        # Kumpulkan harga dari kolom 2-8 (7 hari)
        prices = []
        for col_idx in range(2, 9):  # kolom 2 sampai 8
            try:
                val = row[col_idx]
                if val and val != '-' and str(val).lower() != 'nan':
                    # Konversi ke float
                    price = float(str(val).replace(',', ''))
                    prices.append(price)
            except:
                pass

        # Hitung rata-rata harga
        if prices:
            avg_price = sum(prices) / len(prices)

            # Buat data entry dengan semua bulan (isi bulan current dengan avg, sisanya 0)
            province_data = {
                'province_code': province_code,
                'province_name': province_name.title(),
                'jan': 0, 'feb': 0, 'mar': 0, 'apr': 0,
                'may': 0, 'jun': 0, 'jul': 0, 'aug': 0,
                'sep': 0, 'oct': 0, 'nov': 0, 'dec': 0
            }

            # Set current month
            province_data[current_month] = round(avg_price, 2)

            transformed_data.append(province_data)
            print(f"  ✓ {province_name} → {province_code}: Rp {avg_price:,.0f}")

    # Buat struktur final
    result = {
        current_year: {
            'metadata': {
                'source': 'Bank Indonesia - Harga Pangan',
                'commodity': 'Beras (Semua Kualitas)',
                'unit': 'Rupiah per Kg',
                'last_update': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'data_type': 'daily_average',
                'period': f'{current_month} {current_year}',
                'note': 'Data harga harian dirata-rata untuk bulan berjalan'
            },
            'data': transformed_data
        }
    }

    print(f"\n✓ Transformasi selesai: {len(transformed_data)} provinsi")
    return result

# Jalankan transformasi
if hasil_selenium['success']:
    frontend_data = transform_bi_to_frontend(hasil_selenium)
else:
    print("Selenium scraping gagal, tidak bisa transform")
    frontend_data = None

Processing 35 rows from BI data...
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8]
  ✓ ACEH → 11: Rp 15,217
  ✓ SUMATERA UTARA → 12: Rp 14,850
  ✓ SUMATERA BARAT → 13: Rp 18,300
  ✓ RIAU → 14: Rp 16,350
  ✓ KEPULAUAN RIAU → 21: Rp 15,470
  ✓ JAMBI → 15: Rp 15,400
  ✓ BENGKULU → 17: Rp 15,450
  ✓ SUMATERA SELATAN → 16: Rp 15,630
  ✓ KEPULAUAN BANGKA BELITUNG → 19: Rp 15,150
  ✓ LAMPUNG → 18: Rp 14,850
  ✓ BANTEN → 36: Rp 14,950
  ✓ JAWA BARAT → 32: Rp 14,900
  ✓ DKI JAKARTA → 31: Rp 16,550
  ✓ JAWA TENGAH → 33: Rp 14,943
  ✓ DI YOGYAKARTA → 34: Rp 14,200
  ✓ JAWA TIMUR → 35: Rp 14,650
  ✓ BALI → 51: Rp 15,400
  ✓ NUSA TENGGARA BARAT → 52: Rp 13,907
  ✓ NUSA TENGGARA TIMUR → 53: Rp 15,300
  ✓ KALIMANTAN BARAT → 61: Rp 16,312
  ✓ KALIMANTAN SELATAN → 63: Rp 17,840
  ✓ KALIMANTAN TENGAH → 62: Rp 17,770
  ✓ KALIMANTAN TIMUR → 64: Rp 16,067
  ✓ KALIMANTAN UTARA → 65: Rp 17,420
  ✓ GORONTALO → 75: Rp 15,280
  ✓ SULAWESI SELATAN → 73: Rp 14,260
  ✓ SULAWESI TENGGARA → 74: Rp 15,450
  ✓ SULAWESI TENGAH → 

## 10. Simpan Data ke Frontend

In [12]:
# Simpan data yang sudah ditransform ke frontend
if frontend_data:
    # Simpan ke frontend directory
    frontend_file = "../frontend/data-harga-beras-bi.json"

    with open(frontend_file, 'w', encoding='utf-8') as f:
        json.dump(frontend_data, f, ensure_ascii=False, indent=2)

    print(f"✓ Data berhasil disimpan ke: {frontend_file}")
    print(f"  Format: {list(frontend_data.keys())}")
    print(f"  Total provinsi: {len(frontend_data['2026']['data'])}")

    # Juga simpan raw data untuk backup
    backup_file = f"../data/bi-harga-beras-raw-{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(backup_file, 'w', encoding='utf-8') as f:
        json.dump(hasil_selenium, f, ensure_ascii=False, indent=2)

    print(f"✓ Raw data backup: {backup_file}")

    # Preview data
    print("\nPreview data (3 provinsi pertama):")
    for i, prov in enumerate(frontend_data['2026']['data'][:3]):
        print(f"  {prov['province_code']} - {prov['province_name']}: Rp {prov['feb']:,.0f}")
else:
    print("Tidak ada data untuk disimpan")

✓ Data berhasil disimpan ke: ../frontend/data-harga-beras-bi.json
  Format: ['2026']
  Total provinsi: 34
✓ Raw data backup: ../data/bi-harga-beras-raw-20260219_213743.json

Preview data (3 provinsi pertama):
  11 - Aceh: Rp 15,217
  12 - Sumatera Utara: Rp 14,850
  13 - Sumatera Barat: Rp 18,300


## 11. Ringkasan & Integrasi Frontend

✅ **Data Scraping Berhasil!**

### Hasil Scraping:
- **Source:** Bank Indonesia - Harga Pangan
- **Komoditas:** Beras (Semua Kualitas)  
- **Periode:** 11-19 Februari 2026 (7 hari)
- **Data:** 34 provinsi dengan harga harian rata-rata

### File Output:
1. **Frontend:** `frontend/data-harga-beras-bi.json` (siap dipakai web)
2. **Backup:** `data/bi-harga-beras-raw-*.json` (raw selenium result)

### Integrasi ke Frontend:
✅ File `frontend/app.js` sudah diupdate untuk:
- Load data dari `data-harga-beras-bi.json`
- Transform data ke format Economic Index (IPE)
- Tampilkan harga beras per provinsi di map

### Testing:
Buka web di browser dan pilih **Economic/Price Heatmap** tab untuk melihat data harga beras.

---

**Next Steps untuk Production:**
1. Setup scheduled scraping (cron job) untuk update data harian
2. Implementasi backend `bi-scraper-service` dengan kode dari notebook ini
3. Add error handling & monitoring
4. Expand komoditas (cabai, bawang, dll)

## 5. Eksplorasi Data Hasil Scraping

In [ ]:
# Tampilkan struktur data
if hasil_scraping['success'] and hasil_scraping['data']:
    print("STRUKTUR DATA HASIL SCRAPING:")
    print("="*70)

    for item in hasil_scraping['data']:
        table_idx = item['table_index']
        data_records = item['data']

        print(f"\nTable #{table_idx + 1}:")
        print(f"  Jumlah record: {len(data_records)}")

        if data_records:
            print(f"  Keys: {list(data_records[0].keys())}")
            print(f"\n  Sample data (3 baris pertama):")

            df_sample = pd.DataFrame(data_records[:3])
            print(df_sample.to_string())
else:
    print("Tidak ada data untuk ditampilkan")

## 6. Konversi ke DataFrame Gabungan

In [ ]:
# Gabungkan semua data ke satu DataFrame
if hasil_scraping['success'] and hasil_scraping['data']:
    all_records = []

    for item in hasil_scraping['data']:
        for record in item['data']:
            # Tambahkan informasi source table
            record['source_table'] = item['table_index']
            all_records.append(record)

    if all_records:
        df_beras = pd.DataFrame(all_records)

        print(f"DataFrame Gabungan Berhasil Dibuat")
        print(f"Shape: {df_beras.shape}")
        print(f"\nColumns: {list(df_beras.columns)}")
        print(f"\nInfo:")
        print(df_beras.info())
        print(f"\nSample data (10 baris pertama):")
        print(df_beras.head(10))
    else:
        print("Tidak ada record untuk dikonversi")
        df_beras = None
else:
    print("Scraping gagal, tidak ada data untuk dikonversi")
    df_beras = None

## 7. Analisis Data Beras

In [ ]:
# Analisis data jika berhasil di-scrape
if df_beras is not None and not df_beras.empty:
    print("ANALISIS DATA BERAS:")
    print("="*70)

    # Cari kolom yang berisi harga
    price_columns = [col for col in df_beras.columns if any(keyword in str(col).lower() for keyword in ['harga', 'price', 'nilai'])]
    print(f"\nKolom harga yang terdeteksi: {price_columns}")

    # Cari kolom yang berisi tanggal/periode
    date_columns = [col for col in df_beras.columns if any(keyword in str(col).lower() for keyword in ['tanggal', 'date', 'bulan', 'month', 'periode', 'tahun'])]
    print(f"Kolom tanggal/periode yang terdeteksi: {date_columns}")

    # Cari kolom yang berisi lokasi
    location_columns = [col for col in df_beras.columns if any(keyword in str(col).lower() for keyword in ['provinsi', 'daerah', 'kota', 'kabupaten', 'wilayah', 'lokasi'])]
    print(f"Kolom lokasi yang terdeteksi: {location_columns}")

    # Tampilkan statistik deskriptif untuk kolom numerik
    print("\nStatistik Deskriptif:")
    print(df_beras.describe())

    # Hitung jumlah data per jenis beras (jika ada kolom jenis/kualitas)
    komoditas_cols = [col for col in df_beras.columns if 'komoditas' in str(col).lower()]
    if komoditas_cols:
        print(f"\nDistribusi per Jenis Beras:")
        print(df_beras[komoditas_cols[0]].value_counts())
else:
    print("Tidak ada data untuk dianalisis")

## 8. Transformasi Data ke Format Frontend

In [ ]:
def transform_to_frontend_format(df):
    """
    Transform data BI ke format yang kompatibel dengan frontend

    Expected output format:
    {
      "2025": {
        "metadata": {...},
        "data": [
          {
            "province_code": "11",
            "province_name": "Aceh",
            "jan": 12500,
            "feb": 12600,
            ...
          }
        ]
      }
    }
    """
    if df is None or df.empty:
        return None

    print("\nMemulai transformasi data ke format frontend...")
    print("="*70)

    # Mapping nama provinsi ke kode (sesuai dengan GeoJSON)
    PROVINCE_CODE_MAPPING = {
        'Aceh': '11', 'Sumatera Utara': '12', 'Sumatra Utara': '12',
        'Sumatera Barat': '13', 'Sumatra Barat': '13', 'Riau': '14',
        'Jambi': '15', 'Sumatera Selatan': '16', 'Sumatra Selatan': '16',
        'Bengkulu': '17', 'Lampung': '18', 'Kepulauan Bangka Belitung': '19',
        'Bangka Belitung': '19', 'Kepulauan Riau': '21', 'DKI Jakarta': '31',
        'Jakarta': '31', 'Jawa Barat': '32', 'Jawa Tengah': '33',
        'DI Yogyakarta': '34', 'Yogyakarta': '34', 'Jawa Timur': '35',
        'Banten': '36', 'Bali': '51', 'Nusa Tenggara Barat': '52',
        'NTB': '52', 'Nusa Tenggara Timur': '53', 'NTT': '53',
        'Kalimantan Barat': '61', 'Kalimantan Tengah': '62',
        'Kalimantan Selatan': '63', 'Kalimantan Timur': '64',
        'Kalimantan Utara': '65', 'Sulawesi Utara': '71',
        'Sulawesi Tengah': '72', 'Sulawesi Selatan': '73',
        'Sulawesi Tenggara': '74', 'Gorontalo': '75', 'Sulawesi Barat': '76',
        'Maluku': '81', 'Maluku Utara': '82', 'Papua': '94',
        'Papua Barat': '91', 'Papua Tengah': '95', 'Papua Pegunungan': '96',
        'Papua Selatan': '97', 'Papua Barat Daya': '92'
    }

    # TODO: Sesuaikan dengan struktur data BI yang sebenarnya
    # Ini adalah template - perlu disesuaikan setelah melihat data aktual

    transformed_data = {}

    print("\n⚠ CATATAN: Fungsi transformasi ini perlu disesuaikan dengan struktur data BI yang sebenarnya")
    print("Silakan modifikasi kode di bawah setelah melihat struktur data hasil scraping")

    return transformed_data

# Jalankan transformasi (jika data tersedia)
if df_beras is not None and not df_beras.empty:
    transformed_data = transform_to_frontend_format(df_beras)
else:
    print("Tidak ada data untuk ditransformasi")
    transformed_data = None

## 9. Simpan Data ke File JSON

In [ ]:
# Simpan raw data hasil scraping
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

if hasil_scraping['success']:
    # Simpan hasil scraping lengkap
    filename_raw = f"../data/bi-scraping-results-{timestamp}.json"
    with open(filename_raw, 'w', encoding='utf-8') as f:
        json.dump(hasil_scraping, f, ensure_ascii=False, indent=2)
    print(f"✓ Raw data disimpan ke: {filename_raw}")

    # Simpan DataFrame jika ada
    if df_beras is not None and not df_beras.empty:
        filename_csv = f"../data/bi-harga-beras-{timestamp}.csv"
        df_beras.to_csv(filename_csv, index=False, encoding='utf-8-sig')
        print(f"✓ DataFrame disimpan ke: {filename_csv}")

        # Simpan versi JSON juga
        filename_json = f"../data/bi-harga-beras-{timestamp}.json"
        df_beras.to_json(filename_json, orient='records', force_ascii=False, indent=2)
        print(f"✓ DataFrame JSON disimpan ke: {filename_json}")

    # Simpan transformed data jika ada
    if transformed_data:
        filename_transformed = f"../frontend/data-harga-beras-bi.json"
        with open(filename_transformed, 'w', encoding='utf-8') as f:
            json.dump(transformed_data, f, ensure_ascii=False, indent=2)
        print(f"✓ Transformed data disimpan ke: {filename_transformed}")
else:
    print("Scraping gagal, tidak ada data untuk disimpan")

## 10. Ringkasan dan Next Steps

In [ ]:
print("\n" + "="*70)
print("RINGKASAN SCRAPING BI HARGA PANGAN")
print("="*70)

if hasil_scraping['success']:
    print(f"\n✓ Status: BERHASIL")
    print(f"  Timestamp: {hasil_scraping['timestamp']}")
    print(f"  Total tables diekstrak: {hasil_scraping['total_tables']}")

    if df_beras is not None:
        print(f"  Total records: {len(df_beras)}")
        print(f"  Columns: {len(df_beras.columns)}")

    print("\nNEXT STEPS:")
    print("  1. Review struktur data hasil scraping")
    print("  2. Sesuaikan fungsi transformasi (cell #8) dengan struktur data aktual")
    print("  3. Transform data ke format frontend yang kompatibel")
    print("  4. Integrasikan dengan frontend app.js (BIPriceService)")
    print("  5. Update backend/services/bi-scraper-service/scraper.py dengan kode ini")
else:
    print(f"\n✗ Status: GAGAL")
    print(f"  Error: {hasil_scraping.get('error', 'Unknown')}")

    print("\nTROUBLESHOOTING:")
    print("  1. Periksa koneksi internet")
    print("  2. Cek apakah website BI.go.id dapat diakses")
    print("  3. Website mungkin memerlukan JavaScript rendering (gunakan Selenium)")
    print("  4. Periksa struktur HTML di browser (F12 Developer Tools)")